In [55]:
import numpy as np
from sklearn import datasets as ds

In [56]:
iris = ds.load_iris() #Load the data set

#Put the data and labels into an x and y matrix
x = iris.data #(150,4)
y = iris.target #(150,1)

#Then normalize the data into a range between 0 & 1
x = x/x.max()
y = y/y.max()


In [57]:
class NeuralNetwork(object):
    def __init__(self):
        np.random.seed(1) #Sets a random seed, this will help when debugging as all the random weights will be the same every time it is run
        
        #Neural Network Model
        self.inputSize = 4 #4 Inputs, sepal length/width and petal length/width
        self.hiddenSize = 3 #Rounded mean of input & output, we'll see how well it works
        self.outputSize = 1 #1 Output to classify which flower it is
        
        #Create the weights randomly into a matrix of the same size as the number of nodes they are connected to 
        self.W1 = np.random.randn(self.inputSize, self.hiddenSize) #input -> hidden
        self.W2 = np.random.randn(self.hiddenSize, self.outputSize) #hidden -> output
        
    def forwardProp(self, x):
        #Propagrate all the data forwards through the network using sigmoid as our activation function
        self.z2 = np.dot(x, self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.W2)
        self.yHat = self.sigmoid(self.z3)
        return self.yHat
    
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))

In [58]:
net = NeuralNetwork()

In [59]:
net.forwardProp(x)

array([[ 0.45588597],
       [ 0.46401394],
       [ 0.46198523],
       [ 0.46406267],
       [ 0.45499112],
       [ 0.44644813],
       [ 0.45803827],
       [ 0.45778889],
       [ 0.46804737],
       [ 0.46427982],
       [ 0.45189199],
       [ 0.45878652],
       [ 0.466218  ],
       [ 0.46860529],
       [ 0.44626116],
       [ 0.43935528],
       [ 0.44632384],
       [ 0.45431375],
       [ 0.44791176],
       [ 0.45041147],
       [ 0.45607893],
       [ 0.4501891 ],
       [ 0.45678161],
       [ 0.45404688],
       [ 0.45896149],
       [ 0.4636741 ],
       [ 0.45465426],
       [ 0.45547564],
       [ 0.45680226],
       [ 0.46215127],
       [ 0.46314379],
       [ 0.45279908],
       [ 0.44915656],
       [ 0.44518236],
       [ 0.46427982],
       [ 0.46046867],
       [ 0.45400965],
       [ 0.46427982],
       [ 0.46646117],
       [ 0.45732174],
       [ 0.45473483],
       [ 0.47526227],
       [ 0.46348289],
       [ 0.45022983],
       [ 0.44906521],
       [ 0